# **Este *ipynb* está enmarcado dentro de un trabajo que busca predecir qué alumnos del departamento de Sistemas la UTN FRBA desertarán.**

Los datos disponibles fueron unificados en una sola tabla denominada **merged_df**.

En este **ipynb** se realizará un análisis estadístico del previamente mencionado dataset.

## **Aclaraciones**

Aquellas líneas de código identificadas con **" # "** fueron **extraídas** de la siguente **fuente**:
* https://github.com/sebajarem/Analisis_desercion_en_ingenieria/tree/master/desercion/diagnostics/00_datos_01


Aquellas líneas de código identificadas con **" ## "** son de **elaboración propia**.

---

## **Importación de librerías**

In [ ]:
## Importamos librerías.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## **Google Colaboratory o Local**
El notebook podrá ser corrido tanto localmente como en Google Colaboratory.

El usuario deberá modificar el root path de acuerdo a su conveniencia.

In [ ]:
## Verificamos si estamos corriendo el noteboock en Google Colaboratory.
var_google_colab = 'google.colab' in str(get_ipython())
print(var_google_colab)

## En el caso de estar en Google Colab, montamos nuestro Drive.
if var_google_colab:
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  ## Direccion root donde está el notebook.
  root_path = "/content/gdrive/MyDrive/Colab Notebooks/GIAR/"

## En el caso de no estar en Google Colab, estamos corriendo localmente el notebook.
else:
  root_path = ""

## **Datasets**

In [ ]:
## Importamos el dataset.
baseline = pd.read_csv(root_path + 'datos/baseline_2009.csv')

## **ANÁLISIS DE LOS DATOS**

In [ ]:
## Imprimimos las dimensiones del dataset.
print(f'Dimensiones del dataset: {baseline.shape}')

In [ ]:
## Imprimimos los nombres de cada columna, el tipo de dato que contiene cada una de ellas y la cantidad de no nulos presentes.
baseline.info()

In [ ]:
baseline['Sexo'].replace(['M','F'],[0,1], inplace = True)

In [ ]:
## Vemos la cantidad y % de NaN por columna.
total = baseline.isnull().sum().sort_values(ascending=False)
percent = round(baseline.isnull().sum()/baseline.isnull().count()*100,2).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent [%]'])
missing_data

**Distribución desertores**

In [ ]:
## Realizamos un gráfico de torta para mostrar la distribucion de los alumnos que desertan y los que no.
proporcion = pd.DataFrame(baseline.deserto.value_counts(normalize=True))
porcentaje = list(set(proporcion['deserto']))

colors = sns.color_palette('pastel')[0:5]
plt.pie(x = porcentaje, labels = ['No desertor','Desertor'], colors = colors, autopct = '%.2f%%')
plt.show()

In [ ]:
## Diferenciamos las variables numéricas de las categóricas.
num_features = ('edad al ingreso', 
                'Ciclo Lectivo de Cursada',
                'Cantidad de veces recursada regular',
                'Descripción de recursada regular_No Recurso', 
                'Descripción de recursada regular_Recurso 1 Vez', 
                'Descripción de recursada regular_Recurso 2 Veces', 
                'Descripción de recursada regular_Recurso 3 Veces', 
                'Descripción de recursada regular_Recurso 4 Veces', 
                'Descripción de recursada regular_Recurso 5 Veces', 
                'Descripción de recursada regular_Recurso n Veces (>5)', 
                'noAprobado', 
                'Aprobado', 
                'Promociono', 
                'Nota', 
                'Nota_max_prom',
                'Turno_Mañana', 
                'Turno_Noche', 
                'Turno_Tarde', 
                'Tipo de aprobación_Cambio Curso', 
                'Tipo de aprobación_Firmo', 
                'Tipo de aprobación_Libre', 
                'Tipo de aprobación_No Firmo', 
                'Tipo de aprobación_Promociono',
                'cantidad de años',
                'Indice_aprobacion')

cat_features = ('EsTecnico', 'deserto', 'Sexo')

In [ ]:
## Obtenemos algunas estadísticas descriptivas de los datos disponibles.
baseline.loc[:, baseline.columns.isin(num_features)].describe().T

In [ ]:
## Determinamos la cantidad de registros con 'edad al ingreso' menor a 17 años.
baseline[baseline['edad al ingreso']<17]['edad al ingreso'].value_counts()

In [ ]:
## Vemos el porcentaje que reprecentan los registros con 'edad al ingreso' menor a 17 años respecto del total.
baseline[baseline['edad al ingreso']<17]['edad al ingreso'].count() / baseline['edad al ingreso'].count() * 100

In [ ]:
## Hay un registro con cantidad de años negativa.
baseline[baseline['cantidad de años']<0]

In [ ]:
## Evidentemente, se trata de un error. Dado que este alumno cursó ya más de 10 materias, estimaremos que se encuentra en su 2do año y modificaremos el dato.
baseline.loc[baseline[baseline['cantidad de años']<0].index,'cantidad de años'] = 2

---
##**Elaboramos gráficos para visualizar la distribución de algunas variables en función de la clase (desertor o no desertor)**

In [ ]:
## Realizamos un gráfico de torta para mostrar la distribucion de los alumnos que desertan y los que no.
proporcion = pd.DataFrame(baseline.deserto.value_counts(normalize=True))
porcentaje = list(set(proporcion['deserto']))

colors = sns.color_palette('pastel')[0:5]
plt.pie(x = porcentaje, labels = ['No desertor','Desertor'], colors = colors, autopct = '%.2f%%')
plt.show()

In [ ]:
## Imprimimos la distribución de la edad de los alumnos al ingresar a la universidad en función de si son o no desertores.
sns.histplot(data=baseline,x=baseline['edad al ingreso'],bins=20,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("edad al ingreso")
plt.title('Distribucion de Edad al Ingreso',size = 20)
plt.show()

In [ ]:
## Calculamos la media de la edad al ingreso en función del outcome de la variable 'deserto'.
baseline.groupby('deserto', as_index=False)['edad al ingreso'].mean()

In [ ]:
## Imprimimos la distribución de 'Cantidad de veces recursada regular' en función de si son o no desertores.
sns.histplot(data=baseline,x=baseline['Cantidad de veces recursada regular'],bins=50,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("Cantidad de veces recursada regular")
plt.title('Distribucion de Cantidad de veces recursada regular',size = 15)
plt.xlim(0,40)
plt.show()

In [ ]:
## Calculamos la media de Cantidad de veces recursada regular en función del outcome de la variable 'deserto'.
baseline.groupby('deserto', as_index=False)['Cantidad de veces recursada regular'].mean()

In [ ]:
## Imprimimos la distribución de 'Promociono' en función de si son o no desertores.
sns.histplot(data=baseline,x=baseline['Promociono'],bins=10,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("Promociono")
plt.title('Distribucion de Promociono',size = 15)
plt.show()

In [ ]:
## Calculamos la media de Promociono en función del outcome de la variable 'deserto'.
baseline.groupby('deserto', as_index=False)['Promociono'].mean()

In [ ]:
## Imprimimos la distribución de 'Aprobado' en función de si son o no desertores.
sns.histplot(data=baseline,x=baseline['Aprobado'],bins=10,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("Aprobado")
plt.title('Distribucion de Aprobado',size = 15)
plt.show()

In [ ]:
## Calculamos la media de Aprobado en función del outcome de la variable 'deserto'.
baseline.groupby('deserto', as_index=False)['Aprobado'].mean()

In [ ]:
## Imprimimos la distribución de 'noAprobado' en función de si son o no desertores.
sns.histplot(data=baseline,x=baseline['noAprobado'],bins=10,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("noAprobado")
plt.title('Distribucion de noAprobado',size = 15)
plt.show()

In [ ]:
## Calculamos la media de noAprobado en función del outcome de la variable 'deserto'.
baseline.groupby('deserto', as_index=False)['noAprobado'].mean()

In [ ]:
## Imprimimos la distribución de 'Indice_aprobacion' en función de si son o no desertores.
sns.histplot(data=baseline,x=baseline['Indice_aprobacion'],bins=10,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("Indice_aprobacion")
plt.title('Distribucion de Indice_aprobacion',size = 15)
plt.show()

In [ ]:
## Calculamos la media de Indice_aprobacion en función del outcome de la variable 'deserto'.
baseline.groupby('deserto', as_index=False)['Indice_aprobacion'].mean()

In [ ]:
## Imprimimos la distribución de 'cantidad de años' en función de si son o no desertores.
sns.histplot(data=baseline,x=baseline['cantidad de años'],bins=6,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("cantidad de años")
plt.title('Distribucion de cantidad de años',size = 15)
plt.show()

In [ ]:
## Calculamos la media de cantidad de años en función del outcome de la variable 'deserto'.
baseline.groupby('deserto', as_index=False)['cantidad de años'].mean()

In [ ]:
## Proporción de mujeres en el dataset.
baseline['Sexo'].value_counts(normalize = True)

In [ ]:
## Proporción desertores entre las mujeres.
baseline[baseline['Sexo'] == 1]['deserto'].value_counts(normalize = True)

In [ ]:
## Proporción desertores entre los hombres.
baseline[baseline['Sexo'] == 0]['deserto'].value_counts(normalize = True)

In [ ]:
## Observamos la distribución de la variable 'Cantidad de veces recursada regular'.
sns.boxplot(y="Cantidad de veces recursada regular", data=baseline)

In [ ]:
## Aquellos valores por encima de 50 los consideramos outliers.
baseline[baseline['Cantidad de veces recursada regular']>30]['Codigo Alumno'].count()

In [ ]:
## Dividimos el dataset.
desertores = baseline[baseline['deserto']==1]
no_desertores = baseline[baseline['deserto']==0]

In [ ]:
## Obtenemos algunas estadísticas descriptivas de la clase desertores.
desertores.loc[:, baseline.columns.isin(num_features)].describe().T

In [ ]:
## Obtenemos algunas estadísticas descriptivas de la clase NO desertores.
no_desertores.loc[:, baseline.columns.isin(num_features)].describe().T

---
##**Descartamos algunos registros**

In [ ]:
## Elimino los registros con nulos.
df = baseline.dropna(how = 'any')

In [ ]:
df.shape

In [ ]:
## Aquellos valores por encima de 30 los consideramos outliers. Eliminaremos dichos registros.
df.drop(df[df['Cantidad de veces recursada regular'] > 30].index, inplace=True)

In [ ]:
df.shape

## **Dataset luego de descartar registros con NaNs y outliers** ####

In [ ]:
## Creamos un dataset sin la variable a predecir y sin 'Código Alumno'.
x = df.loc[:, ~df.columns.isin(('Codigo Alumno','deserto'))].dropna(how='any')

In [ ]:
## Dividimos el dataset.
desertores1 = df[df['deserto']==1]
no_desertores1 = df[df['deserto']==0]

In [ ]:
## Realizamos un gráfico de torta para mostrar la distribucion de los alumnos que desertan y los que no luego de haber descartado registros.
proporcion = pd.DataFrame(df.deserto.value_counts(normalize=True))
porcentaje = list(set(proporcion['deserto']))

colors = sns.color_palette('pastel')[0:5]
plt.pie(x = porcentaje, labels = ['No desertor','Desertor'], colors = colors, autopct = '%.2f%%')
plt.show()

In [ ]:
## Obtenemos algunas estadísticas descriptivas de la clase desertores.
desertores1.loc[:, baseline.columns.isin(num_features)].describe().T

In [ ]:
## Obtenemos algunas estadísticas descriptivas de la clase NO desertores.
no_desertores1.loc[:, baseline.columns.isin(num_features)].describe().T

In [ ]:
## Distribución del sexo en el dataset.
df['Sexo'].value_counts(normalize = True)

In [ ]:
## Proporción desertores entre las mujeres.
df[df['Sexo'] == 1]['deserto'].value_counts(normalize = True)

In [ ]:
## Proporción desertores entre los hombres.
df[df['Sexo'] == 0]['deserto'].value_counts(normalize = True)

In [ ]:
## Elaboramos e imprimimos la matriz de correlación lineal de Pearson entre las features.
sns.set_style("white")
sns.set_context("talk")
sns.set_style("ticks")
plt.subplots(figsize=(11, 11))
sns.heatmap(np.corrcoef(x.T), xticklabels=x.columns, yticklabels=x.columns)
plt.title('Matriz de correlación lineal de Pearson entre las features')
plt.show()

##**OBSERVACIONES**



---

### **Observaciones extraídas de la abajo mencionada fuente:**

* El dataset se encuentra balanceado. **43.88%** de los alumnos del dataset desertó mientras que el **56.12%** sigue en carrera.

* La variable **EsTecnico** tiene un **13.32%** de datos nulos. Dependiendo el método que se use podrá tolerarse o no. En los casos que no se pueda tolerar, se tendrá que imputar algún valor o se podrá optar por descartar la variable.

* El valor mínimo de la variable **edad_al_ingreso** es 11. Un valor muy bajo y puede tratarse de un error. Se analizaron la cantidad de casos que existen en que esta variable tiene un valor menor a 17, que es la mínima edad que podría entrar un estudiante a la universidad respetando todos los ciclos lectivos sin adelantar ninguno de las etapas de estudio anteriores, y dicho valor es de 4 observaciones. Las cuales representan una cantidad insignificante respecto del total de observaciones 4558 (0.08%). Por lo tanto al no poder verificarlo por el momento se decide dejarlo.


### **FUENTE:** 
* https://github.com/sebajarem/Analisis_desercion_en_ingenieria/tree/master/desercion/diagnostics/00_datos_01  (**GIAR 2021** - Romero, G., et al. Predictor de deserción universitaria)
---

---
### **Observaciones a partir del presente trabajo:**


* Hay un registro con **cantidad de años** negativa. Evidentemente, se trata de un error. Dado que ese alumno cursó ya más de 10 materias, estimaremos que se encuentra en su 2do año y modificaremos el dato.

* Luego de remover los registros que tenían valores nulos, podemos afirmar que el dataset sigue balanceado. **41.24%** de los alumnos del dataset desertó, mientras que el **58.76%** sigue en carrera.


* El porcentaje de desertores entre los alumnos de **sexo masculino** es del **42.09%**, mientras que entre los de **sexo femenino** dicho porcentaje desciende al **35.44%**.

---

##**GUARDAMOS EL DATASET**

In [ ]:
## Guardamos el dataframe en un csv.
df.to_csv(root_path + 'datos/base_datos_estudiantes00.csv',index=False)